It's finally time to meet back up with the Elves. When you try to contact them, however, you get no reply. Perhaps you're out of range?

You know they're headed to the grove where the star fruit grows, so if you can figure out where that is, you should be able to meet back up with them.

Fortunately, your handheld device has a file (your puzzle input) that contains the grove's coordinates! Unfortunately, the file is encrypted - just in case the device were to fall into the wrong hands.

Maybe you can decrypt it?

When you were still back at the camp, you overheard some Elves talking about coordinate file encryption. The main operation involved in decrypting the file is called mixing.

The encrypted file is a list of numbers. To mix the file, move each number forward or backward in the file a number of positions equal to the value of the number being moved. The list is circular, so moving a number off one end of the list wraps back around to the other end as if the ends were connected.

For example, to move the `1` in a sequence like `4, 5, 6, 1, 7, 8, 9`, the `1` moves one position forward: `4, 5, 6, 7, 1, 8, 9`. To move the `-2` in a sequence like `4, -2, 5, 6, 7, 8, 9`, the `-2` moves two positions backward, wrapping around: `4, 5, 6, 7, 8, -2, 9`.

The numbers should be moved in the order they originally appear in the encrypted file. Numbers moving around during the mixing process do not change the order in which the numbers are moved.

Consider this encrypted file:

```
1
2
-3
3
-2
0
4
```

Mixing this file proceeds as follows:

```
Initial arrangement:
1, 2, -3, 3, -2, 0, 4

1 moves between 2 and -3:
2, 1, -3, 3, -2, 0, 4

2 moves between -3 and 3:
1, -3, 2, 3, -2, 0, 4

-3 moves between -2 and 0:
1, 2, 3, -2, -3, 0, 4

3 moves between 0 and 4:
1, 2, -2, -3, 0, 3, 4

-2 moves between 4 and 1:
1, 2, -3, 0, 3, 4, -2

0 does not move:
1, 2, -3, 0, 3, 4, -2

4 moves between -3 and 0:
1, 2, -3, 4, 0, 3, -2
```

Then, the grove coordinates can be found by looking at the 1000th, 2000th, and 3000th numbers after the value 0, wrapping around the list as necessary. In the above example, the 1000th number after 0 is 4, the 2000th is -3, and the 3000th is 2; adding these together produces 3.

Mix your encrypted file exactly once. ***What is the sum of the three numbers that form the grove coordinates?***

In [52]:
import numpy as np
input_file  = open("data/2022-12-20-input.txt")
sequence = np.array([int(line.strip()) for line in input_file.readlines()])
len(sequence)
max(abs(np.array(sequence)))

9993

In [78]:
def MixFile(sequence, times = 1):
    sequence_length = len(sequence)
    positions = np.array(range(sequence_length))
    for j in range(times):
        for i in range(sequence_length):
            move = np.sign(sequence[i]) * (abs(sequence[i]) % (sequence_length - 1))
            #move = sequence[i]
            cur_position = positions[i]
            final_position = cur_position + move
            match np.sign(sequence[i]):
                case 1 if final_position >= sequence_length:
                    final_position += 1
                case -1 if final_position <= 0:
                    final_position -= 1
            final_position = final_position % sequence_length
            min_position = min(cur_position, final_position)
            max_position = max(cur_position, final_position)
            positions[np.where((positions >= min_position) & (positions <= max_position))] += (1 if final_position < cur_position else -1)
            positions[i] = final_position
    return positions

def MixedSequence(sequence, positions):
    return np.array([sequence[np.where(positions == x)][0] for x in range(len(sequence))])
    
test_sequence = np.array([1, 2, -3, 3, -2, 0, 4])
test_final_sequence = MixedSequence(test_sequence, MixFile(test_sequence))
print(test_final_sequence)
zero_index = np.where(test_final_sequence == 0)[0][0]
coordinate_sum = (test_final_sequence[(zero_index + 1000) % len(test_sequence)] +
                  test_final_sequence[(zero_index + 2000) % len(test_sequence)] +
                  test_final_sequence[(zero_index + 3000) % len(test_sequence)])
print(f'The sum of the coordinates is {coordinate_sum}')


[ 1  4  2  3 -2  0 -3]
The sum of the coordinates is 3


In [76]:
final_positions = MixFile(sequence)
final_sequence = MixedSequence(sequence, final_positions)
zero_index = np.where(final_sequence == 0)[0][0]
coordinate_sum = (final_sequence[(zero_index + 1000) % len(sequence)] +
                  final_sequence[(zero_index + 2000) % len(sequence)] +
                  final_sequence[(zero_index + 3000) % len(sequence)])
print(f'The sum of the coordinates is {coordinate_sum}')

The sum of the coordinates is 13883


array([    0, -7861,  3030,  4055,  5010, -8594,  5015,    63,  6508,
        4167])

The grove coordinate values seem nonsensical. While you ponder the mysteries of Elf encryption, you suddenly remember the rest of the decryption routine you overheard back at camp.

First, you need to apply the decryption key, 811589153. Multiply each number by the decryption key before you begin; this will produce the actual list of numbers to mix.

Second, you need to mix the list of numbers ten times. The order in which the numbers are mixed does not change during mixing; the numbers are still moved in the order they appeared in the original, pre-mixed list. (So, if -3 appears fourth in the original list of numbers to mix, -3 will be the fourth number to move during each round of mixing.)

Using the same example as above:

```
Initial arrangement:
811589153, 1623178306, -2434767459, 2434767459, -1623178306, 0, 3246356612

After 1 round of mixing:
0, -2434767459, 3246356612, -1623178306, 2434767459, 1623178306, 811589153

After 2 rounds of mixing:
0, 2434767459, 1623178306, 3246356612, -2434767459, -1623178306, 811589153

After 3 rounds of mixing:
0, 811589153, 2434767459, 3246356612, 1623178306, -1623178306, -2434767459

After 4 rounds of mixing:
0, 1623178306, -2434767459, 811589153, 2434767459, 3246356612, -1623178306

After 5 rounds of mixing:
0, 811589153, -1623178306, 1623178306, -2434767459, 3246356612, 2434767459

After 6 rounds of mixing:
0, 811589153, -1623178306, 3246356612, -2434767459, 1623178306, 2434767459

After 7 rounds of mixing:
0, -2434767459, 2434767459, 1623178306, -1623178306, 811589153, 3246356612

After 8 rounds of mixing:
0, 1623178306, 3246356612, 811589153, -2434767459, 2434767459, -1623178306

After 9 rounds of mixing:
0, 811589153, 1623178306, -2434767459, 3246356612, 2434767459, -1623178306

After 10 rounds of mixing:
0, -2434767459, 1623178306, 3246356612, -1623178306, 2434767459, 811589153
The grove coordinates can still be found in the same way. Here, the 1000th number after 0 is 811589153, the 2000th is 2434767459, and the 3000th is -1623178306; adding these together produces 1623178306.
```

Apply the decryption key and mix your encrypted file ten times. ***What is the sum of the three numbers that form the grove coordinates?***

In [84]:
decryption_key = 811589153
actual_test_sequence = test_sequence * decryption_key
actual_test_sequence
MixedSequence(actual_test_sequence, MixFile(actual_test_sequence, 10))

array([          0, -2434767459,  1623178306,  3246356612, -1623178306,
        2434767459,   811589153])

In [85]:
actual_sequence = sequence * decryption_key
actual_final_positions = MixFile(actual_sequence, 10)
actual_final_sequence = MixedSequence(actual_sequence, actual_final_positions)
actual_zero_index = np.where(actual_final_sequence == 0)[0][0]
coordinate_sum = (actual_final_sequence[(actual_zero_index + 1000) % len(actual_sequence)] +
                  actual_final_sequence[(actual_zero_index + 2000) % len(actual_sequence)] +
                  actual_final_sequence[(actual_zero_index + 3000) % len(actual_sequence)])
print(f'The sum of the coordinates is {coordinate_sum}')

The sum of the coordinates is 19185967576920
